# Loading library

In [109]:
import pandas as pd
import numpy as np
import warnings
# Suppress RuntimeWarning: divide by zero encountered in divide
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Xử lý đầu vào

1. Về ràng buộc đẳng thức, bất đẳng thức --> chuyển về dạng chuẩn  
- nếu <=: giữ nguyên  
- nếu >=: nhân với -1  
- nếu =: nhân vế trái với -1  

2. Về ràng buộc dấu của biến 
- x>=0: giữ nguyên
- x<=0: thay x=-x, ghi nhớ vị trí biến để kết luận x=-x
- x không có ràng buộc: tạo 2 biến +x, -x mới, ghi nhớ vị trí biến để kết luận x = (+x) - (-x)

3. Về hướng của hàm mục tiêu  
- Nếu optimize_direction là max thì đưa hàm mục tiêu f(x) thành -f(-x)
- Nếu optimize_direction là min thì giữ nguyên

In [123]:
def processing_input(A,B):
    """ 
    # goal
    - chuyển 2 dataframe lưu ràng buộc đẳng thức/bất đẳng thức và ràng buộc về dấu của biến trong bài toán quy hoạch tuyến tính cho trước thành những ma trận có thể tính toán, xử lý được

    # input
    - A (dataframe): lưu ràng buộc đẳng thức/bất đẳng thức
    - B (dataframe): ràng buộc về dấu của biến 

    # output
    - optimize_direction: hướng của hàm mục tiêu
    - norm_arr (np.array): ma trận của bài toán sau khi chuyển về dạng chuẩn (P')
    - tab_arr (np.array): ma trận tương ứng với dạng bảng của phương pháp đơn hình của norm_arr
    - rsby (np.array): ma trận lưu trữ cách biểu diễn nghiệm của (P) theo (P')
    """

    # 1. Kiểm tra hướng của hàm mục tiêu
    A.iloc[0] = A.iloc[0].fillna(0)
    optimize_direction = ''
    if A.iloc[0,-1] == 'max':
        optimize_direction = 'max'
    else:
        optimize_direction = 'min'
    A.iloc[0,-1] = 0
    # đưa cột cuối cùng của A về dạng numerical
    A[A.shape[1]-1] = pd.to_numeric(A[A.shape[1]-1])
    # nếu optimize_direction là max thì đưa hàm mục tiêu f(x) thành -f(-x)
    if optimize_direction == 'max':
        A.iloc[0,:] = -A.iloc[0,:]

    # 2. Xử lý ràng buộc về đẳng thức và bất đẳng thức
    # norm_arr là ma trận mới sau khi chuẩn hoá ma trận A. Dùng để làm biến đầu vào trong hàm xoay Bland
    A.iloc[0] = A.iloc[0].fillna(0)
    norm_arr = A.drop(columns=A.columns[A.shape[1] -1 -1]).to_numpy()
    nrow, ncol = norm_arr.shape
    for i in range(1,nrow):
        if A.iloc[i, -2] == ">=":
            norm_arr[i] = -norm_arr[i]
        elif A.iloc[i, -2] == "=":
            norm_arr[i, :-1] = -norm_arr[i, :-1]

    # 3. Xử lý ràng buộc về dấu của biến
    # rsby là hàm result by vars: dùng để kết luận biến cũ theo biến mới sau khi chạy xong thuật toán xoay Bland
    rsby = np.eye(B.shape[1])
    k = 0
    for j in range(B.shape[1]):
        if B.iloc[0, j] == "<=":
            norm_arr[:, k] = -norm_arr[:,k]
            rsby[:, k] = -rsby[:,k]
        elif B.iloc[0, j] == 0:
            new_col_norm_A = -norm_arr[:, k]
            norm_arr = np.insert(norm_arr, k+1, new_col_norm_A, axis = 1)
            new_col_rsby = -rsby[:, k]
            rsby = np.insert(rsby, k+1, new_col_rsby, axis = 1)
            k = k+1
        k = k+1

    # 4. đưa norm_arr về dạng bảng của phương pháp đơn hình tab_arr
    # Đưa về dạng ma trận để có thể giải bài toán x1|x2|...|xn|w1|w2|...|wn|bi 
    n_var = norm_arr.shape[1] - 1
    n_const = norm_arr.shape[0] -1
    arr = np.insert(np.zeros((1, n_const)), 1, np.eye(n_const), axis = 0)
    tab_arr = np.hstack((norm_arr[:, :n_var], arr, norm_arr[:, n_var:]))
    # cho tab_arr[1:,:-1] = -tab_arr[1:,:-1]
    tab_arr[1:,:-1] = -tab_arr[1:,:-1]
    
    return(norm_arr, tab_arr, rsby, optimize_direction)

Lấy ví dụ ở câu 2.5d/63 sách QHTT

In [124]:
A = pd.read_csv("input_const.csv", header=None)
B = pd.read_csv('input_vars.csv', header=None)

- A là ma trận chứa các ràng buộc về đẳng thức và bất đẳng thức  

z = min  5x1 - 10x2  
-2x1 +   x2  <= 1  
x1 - x2 >= -2  
3x1 + x2 <= 8
-2x1 + 3x2 >= -9  
4x1 + 3x2 >= 0

5,-10,,min  
-2,1,<=,1  
1,-1,>=,-2  
3,1,<=,8  
-2,3,>=,-9  
4,3,>=,0  

- B là ma trận chứa các ràng buộc về dấu của biến
- x1 >= 0
- x2 tự do

.>=,0


In [125]:
A

,0,1,2,3
0,5,-10,NaN,min
1,-2,1,<=,1
2,1,-1,>=,-2
3,3,1,<=,8
4,-2,3,>=,-9
5,4,3,>=,0


In [126]:
B

,0,1
0,>=,0


In [131]:
norm_A, tab_arr, rsby, optimize_direction = processing_input(A,B)

# Bland Algorithm

Áp dụng bland khi hàm mục tiêu có biến âm, bi >= 0

Số ràng buộc và số biến ban đầu của bài toán quy hoạch tuyến tính 

In [115]:
n_const = tab_arr.shape[0] - 1
n_var = tab_arr.shape[1] - tab_arr.shape[0] - 1

Xoay bland


In [116]:
def select_input_output_index(arr):
    """
    # goal 
    - tìm chỉ số dòng i, chỉ số cột j của biến cần xoay trong thuật toán Bland

    # input
    - arr (np.array): ma trận tương tự dạng bảng của phương pháp đơn hình

    # output
    - nếu i = None, tức là không có biến vào --> từ vựng tối ưu
    - nếu i != None, j = None, tức là có biến vào, không có biến ra --> bài toán không giới nội
    - nếu i != None, j != None, trả về chỉ số (i,j) cần tìm
    """
    # Find the index of the first negative element, or None if no negative elements exist.
    const_arr = arr[0,:-1]
    lst = const_arr[const_arr<0]
    ind_in = None
    ind_out = None
    if len(lst) != 0:
        ind_in = np.where(const_arr<0)[0][0]
    else:
        # Tu vung toi uu
        return('TVTU')
    
    # Find the index of the largest negative number 
    lst = arr[1:, -1]/[i if i<0 else np.nan for i in arr[1:, ind_in]]
    if len(lst[lst<=0]) == 0:
        ind_out = None
    else:
        ind_out = np.where(lst == max(lst[lst<=0]))[0][0] + 1

    # Bai toan Khong gioi noi
    if ind_in != None and ind_out == None:
        return('KGN') 
    
    # Tra ve vi tri cua bien can xoay
    return(ind_in, ind_out)
    

def find_eye_list(arr_col):
    """ 
    # goal
    - Tìm vị trí dòng của biến cơ sở ở từ vựng tối ưu. Dùng để kết luận nghiệm (xét ở trường hợp duy nhất nghiệm)
    - Ở từ vựng tối ưu, nếu x_i là biến cơ sở, thì cột x_i - arr_col là mảng chỉ có số 0 và -1. Và có duy nhất một số -1 
    
    # input
    - arr_col: ma trận cột của từ vựng tối ưu

    # output
    - vị trí dòng của biến cơ sở
    """
    # Finds the index of the unique element equal to -1 in a binary array.
    lst = arr_col.tolist()
    for element in lst:
        if element not in [0,-1]:
            return None
    if lst.count(-1) == 1:
        return lst.index(-1)

def bland_rotate(arr, i, j):
    """
    # goal
    - sử dụng thuật toán bland để xoay ma trận arr thành một ma trận mới arr_new

    # input
    - arr: mảng cần xoay
    - i: vị trí hàng của biến cần xoay
    - j: vị trí cột của biến cần xoay

    # output: return về mảng mới vừa được xoay
    """
    nrow, ncol = arr.shape
    arr_new = arr.copy()
    arr_new[i] = arr_new[i]/(-arr[i,j])
    for row in range(nrow):
        if row != i:
            for col in range(ncol):
                if col == j:
                    arr_new[row, col] = 0
                else:
                    arr_new[row, col] = arr[row, j] * arr_new[i, col] + arr[row, col]
    
    return arr_new

def check_type_result(arr_tvtu):
    """ 
    # goal
    - Tìm các trường hợp xảy ra khi kết luận nghiệm

    # input
    - arr_tvtu (np.array): ma trận từ vựng tối ưu của bài toán

    # output
    type_result: các trường hợp khi dùng bland
    - DDN: duy nhất nghiệm
    - KGN: không giới nội
    - VSN: vô số nghiệm
    - VN: vô nghiệm (thường xảy ra ở từ cuối pha 1 của thuật toán 2 pha)
    """
    tvtu = arr_tvtu[0, :-1]
    # lấy được trường hợp không giới nội KGN hoặc duy nhất nghiệm DNN
    type_result = select_input_output_index(arr_tvtu)
    if type_result == "TVTU":
        if np.count_nonzero(tvtu) < (arr_tvtu.shape[1] - arr_tvtu.shape[0] - 1):
            type_result = "VSN"
        else:
            type_result = "DNN"
    return type_result

def print_result(rsby, optimize_direction, df_result):
    """ 
    # input
    - rsby: ma trận biểu diễn của 1 biến
    - optimize_direction: biến lưu hướng tối ưu của hàm mục tiêu (max or min)
    - df_result: list lưu các bước giải của bài toán QHTT

    # output
    - show các bước giải, nếu lựa chọn show
    - show kết quả cuối cùng: 
        + btoan có nghiêm duy nhất: show nghiêm tối ưu, giá trị tối ưu
        + btoan vô nghiệm: nếu optimize_direction là max thì giá trị tối ưu là -inf, ngược lại
        + btoan không giới nội: nếu optimize_direction là max thì giá trị tối ưu là inf, ngược lại
        + btoan vô số nghiệm: cho các biến không cơ sở xuất hiện ở hàm mục tiêu bằng 0, viết biến cơ sở theo điều kiện các biến không cơ sở
    """

    



Giải bài toán bằng cách dùng thuật toán Bland, và kết luận nghiệm tối ưu, giá trị tối ưu của bài toán

PHẦN NÀY CHƯA CODE XONG NHA, MỚI SHOW KẾT QUẢ CHO PHẦN DUY NHẤT NGHIỆM  

In [130]:
# tạo danh sách df_result để lưu lại các từ vựng tại mỗi lần xoay
df_result = list()
nrow, ncol = tab_arr.shape

# thêm từ vựng ban đầu vào danh sách
cur_arr = tab_arr
df_result.append(cur_arr)

# type_result: các trường hợp xảy ra khi kết luận nghiệm: 
# DDN: duy nhất nghiệm, KGN: không giới nội, VN: vô nghiệm, VSN: vô số nghiệm

type_result = None
# result_x_P = None
# z = None

# Xoay bland cho đến khi tìm được từ vựng tối ưu
in_out_ind = select_input_output_index(cur_arr)
while in_out_ind != 'TVTU':
    if in_out_ind != "KGN":
        j, i = in_out_ind
        new_arr = bland_rotate(cur_arr, i, j)
        df_result.append(new_arr)
    else:
        type_result = "KGN"
        break
    cur_arr = new_arr
    in_out_ind = select_input_output_index(cur_arr)

# print result
type_result = check_type_result(df_result[-1])

# tạo mảng lưu giá trị nghiệm tối ưu của (P)
result_x_P = None
# 
z = None

# 1. DNN
if type_result == "DNN":
    arr_last = df_result[-1]
    result = np.zeros(arr_last.shape[1])
    result[-1] = arr_last[0, -1]
    for col in range(ncol-1):
        ind = find_eye_list(arr_last[:,col])
        if ind != None:
            result[col] = arr_last[ind, -1]
        else:
            result[col] = 0
    
    # result_x_P_hat là ma trận chứa nghiệm của các biến x1, ..., xn của (P')
    result_x_P_hat = result[:(len(result) - n_const - 1)]
    # result_x_P_hat là ma trận chứa nghiệm của các biến x1, ..., xn của (P)
    result_x_P = rsby@result_x_P_hat
    z = None
    if optimize_direction == 'min':
        z = result[-1]
    else:
        z = -result[-1]

    #
    print(f"x = {result_x_P}, z = {z}")
    #
    

# 2. KGN
elif type_result == "KGN":
    result_x_P = "Bai toan khong gioi noi"
    if optimize_direction == 'min':
        z = -np.inf
    else:
        z = np.inf
# # 3. VSN
# elif type_result == "VSN":

# print(f"x = {result_x_P}, z = {z}")
print(f"\nStep by step: ")
for result in df_result:
    print(result, end = "\n\n")

x = [1.5 3.5], z = -27.5

Step by step: 
[[  5. -10.  10.   0.   0.   0.   0.   0.   0.]
 [  2.  -1.   1.  -1.  -0.  -0.  -0.  -0.   1.]
 [  1.  -1.   1.  -0.  -1.  -0.  -0.  -0.   2.]
 [ -3.  -1.   1.  -0.  -0.  -1.  -0.  -0.   8.]
 [ -2.   3.  -3.  -0.  -0.  -0.  -1.  -0.   9.]
 [  4.   3.  -3.  -0.  -0.  -0.  -0.  -1.   0.]]

[[-15.   0.   0.  10.   0.   0.   0.   0. -10.]
 [  2.  -1.   1.  -1.  -0.  -0.  -0.  -0.   1.]
 [ -1.   0.   0.   1.  -1.   0.   0.   0.   1.]
 [ -5.   0.   0.   1.   0.  -1.   0.   0.   7.]
 [  4.   0.   0.  -3.  -0.  -0.  -1.  -0.  12.]
 [ 10.   0.   0.  -3.  -0.  -0.  -0.  -1.   3.]]

[[  0.   0.   0.  -5.  15.   0.   0.   0. -25.]
 [  0.  -1.   1.   1.  -2.   0.   0.   0.   3.]
 [ -1.   0.   0.   1.  -1.   0.   0.   0.   1.]
 [  0.   0.   0.  -4.   5.  -1.   0.   0.   2.]
 [  0.   0.   0.   1.  -4.   0.  -1.   0.  16.]
 [  0.   0.   0.   7. -10.   0.   0.  -1.  13.]]

[[  0.     0.     0.     0.     8.75   1.25   0.     0.   -27.5 ]
 [  0.    -1.     1.   

## 2 pha

- input_consts  
-1,-3,-1,,max  
2,-5,1,<=,-5  
2,-1,2,<=,4  
- input_vars  
>=,>=,>=

In [122]:
C =  pd.read_csv("input_consts2.csv", header=None)
D = pd.read_csv('input_vars2.csv', header=None)
new_arr, tab_arr, rsby, optimize_direction = processing_input(C, D)
tab_arr

array([[-1., -3., -1.,  0.,  0.,  0.],
       [-2.,  5., -1., -1., -0., -5.],
       [-2.,  1., -2., -0., -1.,  4.]])

Kiểm tra xem có tồn tại bi<0 ở tab_arr không. Nếu có thì mới áp dụng thuật toán 2 pha

In [ ]:
np.any(tab_arr[1:,-1]<0)

True

### Pha 1 
Lập bài toán bổ trợ  
- Thêm biến x0 vào ràng buộc đẳng thức, bất đẳng thức. Lập từ vựng xuất phát C = x0  
- Đưa về dạng ma trận để có thể giải, lưu vào biến tab_C (np.array)

In [ ]:
if np.any(tab_arr[1:,-1]<0):
    tab_C = tab_arr.copy()
    tab_C = np.insert(tab_C, 0, np.ones(tab_C.shape[0]), axis = 1)
    tab_C[0, 1:] = 0
    print(tab_C)


[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1. -2.  5. -1.  1. -1. -0. -5.]
 [ 1. -2.  1. -2.  2. -0. -1.  4.]]


Xoay từ vựng đầu tiên  
- Chọn biến vào: x0
- Chọn biến ra: biến ở dòng ứng với bi âm nhất, ưu tiên dòng có hệ số nhỏ nhất  
- Xoay từ vựng như đơn hình 

In [ ]:
# chọn biến vào
j = 0
# chọn biến ra, do ở thuật toán 2 pha, luôn tồn tại bi<0 nên luôn tồn tại i
bi = tab_C[1:, -1]
i = np.where(bi == np.min(bi[bi<0]))[0][0] + 1
# xoay từ vựng như đơn hình

In [ ]:
df_result = list()
df_result.append(tab_C)
new_arr = bland_rotate(tab_C,i,j)
df_result.append(new_arr)
new_arr


array([[ 0.,  2., -5.,  1., -1.,  1.,  0.,  5.],
       [-1.,  2., -5.,  1., -1.,  1.,  0.,  5.],
       [ 0.,  0., -4., -1.,  1.,  1., -1.,  9.]])

Từ từ vựng thứ 2, áp dụng cách xoay đơn hình --> từ vựng cuối pha 1

Từ vựng cuối pha 1 (từ vựng tối ưu của P1), xét hàm mục tiêu
- Nếu hàm mục tiêu chỉ có biến x0 --> chuyển sang pha 2
- Nếu hàm mục tiêu gồm biến x0 và những biến khác (hoặc không có x0) --> kết luận bài toán vô nghiệm --> dừng thuật toán. 

### Pha 2